# Fitting the $f\sigma_8(z)$ curve of a coasting universe with positive curvature to $f\sigma_8$ values derived from RSD measurements

In [ ]:
#Importing relevant packages
import numpy as np
import emcee
from scipy.optimize import minimize
from numpy import genfromtxt
from scipy.integrate import quad
from scipy.special import kn
from tqdm import tqdm

## Recalibration/homogenization of the $f\sigma_8$ data

In order to fit a given model's $f\sigma_8(z)$ curve to the dataset, the latter must be recalibrated to match the model, since the data were obtained under the assumption of a fiducial cosmology which is different from the model. The recalibration involves the calculation of a correction factor, $q(z, \Omega_\mathrm{m,0}, \Omega^\mathrm{fid}_\mathrm{m,0}) = \frac{H(z)D_\mathrm{A}(z)}{H^\mathrm{fid}(z)D^\mathrm{fid}_{\mathrm{A}}(z)}$, with which the $f\sigma_8(z)$ values and their uncertainties must be multiplied. For this model, we have 
- $H^\mathrm{fid}(z)=H^\mathrm{fid}_0 \sqrt{\Omega^\mathrm{fid}_{\mathrm{m,0}}(1+z)^3+\Omega^\mathrm{fid}_{k,0}(1+z)^2+\Omega^\mathrm{fid}_{\Lambda,0}}$
- $D^\mathrm{fid}_A(z)=\frac{c}{H^\mathrm{fid}_0}\frac{1}{1+z}\int_{0}^{z}\frac{H^\mathrm{fid}_0}{ H^\mathrm{fid}(z’)}\mathrm{d} z’$
- $H(z)=H_0(1+z)$
- $D_\mathrm{A}(z)=\frac{c}{H_0}\frac{1}{1+z}|\sin{(\ln{(1+z)})}|$


In [ ]:
#Importing data file
originaldata = genfromtxt('fsigma_8_dataset.csv', delimiter=',')
z=originaldata[:,0]
fs8=originaldata[:,1]
dfs8=originaldata[:,2]
om=originaldata[:,3]
sigfigs=originaldata[:,4]

#Initialising converted arrays for recalibrated data points
l=len(originaldata)
FS8=np.zeros(l)
dFS8=np.zeros(l)

#Recalibrating the original data set for a coasting cosmology with positive curvature
for i in range(l):
    
    hlin=1+z[i] #hubble parameter in a coasting universe
   
    dalin=np.abs(np.sin(np.log(1+z[i]))) #angular diameter distance in a coasting universe #angular diameter distance in a coasting universe with positive curvature
  
    Om=om[i] #Omega_m0 of the fiducial cosmology of the current data point
    
    hfid=np.sqrt(Om*(1+z[i])**3+1-Om) #Hubble parameter in the fiducial cosmology
   
    def integrand(x, omega_m0):
        return 1/(np.sqrt(omega_m0*(1+x)**3+1-omega_m0)) #integrand of the fiducial angular diameter distance integral
    
    dafid=quad(integrand, 0, z[i], args=(Om))[0] #fiducial angular diameter distance
    
    #note that those terms which will cancel in 'correction', 
    # namely c/(1+z), and the various Hubble parameters H_0, are omitted above, as they are irrelevant
    
    correction=(hlin/hfid)*(dalin/dafid)
    
    FS8[i]=fs8[i]*correction
    
    dFS8[i]=dfs8[i]*correction

    
#Rounding to corresponding number of significant figures
FS8r=np.zeros(l)
dFS8r=np.zeros(l)
for i in range(l):
        nos=int(sigfigs[i]) #"nos" refers to "number of significant (figures)"
        FS8r[i]=np.around(FS8[i],decimals=nos)
        dFS8r[i]=np.around(dFS8[i],decimals=nos)

#outmatrix=np.array([z,FS8r,dFS8r]).T
#outmatrix.reshape(len(z),3)
#np.savetxt("recalibrated_data_positive_curvature_coasting.csv", outmatrix, delimiter=",")

## Curve fitting using the emcee sampler

### Defining the log-likelihood function, the priors, and the log-probability functions

In [ ]:

def log_likelihood(theta, z, FS8, dFS8):
    q, s = theta
    model = -s/kn(1,np.sqrt(6*q))*(0.5*np.sqrt(1+z)*kn(1,np.sqrt(6*q*(1+z))) - 1/4*np.sqrt(6*q)*(1+z)*(kn(0,np.sqrt(6*q*(1+z)))+kn(2,np.sqrt(6*q*(1+z)))))
    sigma2 = dFS8**2
    return -0.5 * np.sum((FS8 - model) ** 2 / sigma2 + np.log(sigma2))

def log_prior(theta):
    q, s= theta
    if 0 < q < 2 and 0.0 < s < 2:
        return 0.0
    return -np.inf

def log_probability(theta, z, FS8, dFS8):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, z, FS8, dFS8)

### Obtaining a position in parameter space from which to start the sampler by computing the maximum likelihood estimates of the parameters $\Omega_\mathrm{m,0}$ and $\sigma_8$.

In [ ]:
nll = lambda *args: -log_likelihood(*args)
initial = np.array([1, 1])
bnds = ((0.000000000000000000000000000001, 2), (0, 2))
soln = minimize(nll, initial, args=(z, FS8r, dFS8r), bounds=bnds)
q_ml, s_ml = soln.x

print("Maximum likelihood estimates:")
print("q = {0:.5f}".format(q_ml))
print("s = {0:.5f}".format(s_ml))

### Performing the curve fitting

In [ ]:
#Runing the sampler

walkers=500
iters=100000

pos = soln.x + 1e-5 * np.random.randn(walkers, 2)
nwalkers, ndim = pos.shape

sampler = emcee.EnsembleSampler(
    nwalkers, ndim, log_probability, args=(z, FS8r, dFS8r)
)
sampler.run_mcmc(pos, iters, progress=True)

#extracting best fit parameters
tau = sampler.get_autocorr_time()

flat_samples = sampler.get_chain(discard=int((4*np.max(tau))), thin=int((0.5*np.min(tau))), flat=True)
print(flat_samples.shape)
np.savetxt("postive curvature coasting chain.csv", flat_samples,  delimiter = ",") #saved in case one should wish to analyze the chain at a later date

#extracting the best-fit parameters
labels = ["q", "s"]
from IPython.display import display, Math
ndim=2
medians=np.zeros(ndim)
for i in range(ndim):
    mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
    qw = np.diff(mcmc)
    medians[i]=mcmc[1]
    txt = "\mathrm{{{3}}} = {0:.3f}_{{-{1:.3f}}}^{{{2:.3f}}}"
    txt = txt.format(mcmc[1], qw[0], qw[1], labels[i])
    display(Math(txt))


#extracting the best-fit value of S_8 from the posterior distributions
s8vals=flat_samples[:,1]*np.sqrt(flat_samples[:,0]/0.3)
np.mean(s8vals)
s8percentiles = np.percentile(s8vals, [16, 50, 84])
qw = np.diff(s8percentiles)
medians=mcmc[1]
txt = "\mathrm{{{3}}} = {0:.3f}_{{-{1:.3f}}}^{{{2:.3f}}}"
txt = txt.format(s8percentiles[1], qw[0], qw[1], "S8")
display(Math(txt))